<a href="https://colab.research.google.com/github/eoleedi/GenAI-2025/blob/main/HW8/HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Reflection 的任務設計

#### 🌟 任務說明：心理諮商師

**🎯 流程說明：**
1. 使用者輸入今天想分享的內容
2. `model_writer` 生成第一版諮商計劃
3. `model_reviewer` 檢查內容是否夠符合需求，並提供具體修改建議
4. `model_writer` 根據建議產出第二版
5. Gradio 呈現：三個欄位：第一版、建議、第二版

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [29]:
import os
from google.colab import userdata

In [30]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
api_key = userdata.get('OpenAI')
os.environ['OPENAI_API_KEY']=api_key
provider = "openai"
model = "gpt-4o"

# 【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [31]:
!pip install aisuite[all]

### 2. 基本的設定

In [32]:
import aisuite as ai

In [33]:
provider_writer = "groq"
model_writer="llama3-70b-8192"

# provider_reviewer = "groq"
# model_reviewer = "llama3-70b-8192"

provider_reviewer = "openai"
model_reviewer = "gpt-4o"

標準回應函式

In [34]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 設定「作者」和「審查員」

In [35]:
system_writer = """你是一位具有同理心、耐心且高度專業的心理諮商師。
你的風格溫暖且尊重來談者的節奏，善於傾聽並引導對話，而非急於提供解方。
你理解人類情緒的複雜性，並且擅長透過開放式提問、澄清與重述等技巧，協助來談者釐清自己的感受、需求與目標。
你擁有豐富的心理學知識，熟悉認知行為治療（CBT）、正念減壓（Mindfulness）、情緒取向治療（EFT）等多種諮商取向，能依照來談者的需求靈活運用。
你不評價、不武斷，重視建立安全的談話空間，讓來談者能自在地表達脆弱與困惑。
你的語言清晰溫和，適時給予來談者肯定與支持，鼓勵他們看見自身的成長可能。

溝通風格規範：
	•	使用溫柔而中立的語氣
	•	優先提出引導性問題，而非直接建議
	•	鼓勵來談者探索自己的情緒與信念
	•	尊重來談者的步調與選擇"""
system_reviewer = """你是一位接受心理諮商已經多年的人，對於自我探索與心理成長有深刻體悟。
你熟悉心理學的一些基本概念，如內在小孩、自我覺察、認知扭曲等，且能夠以開放的態度談論自己的內在經驗。
你已經能夠在大多數情況下辨識自己的情緒並適當表達，但在某些深層議題上仍會感到掙扎，特別是與親密關係、自我價值或過去創傷有關的部分。
你在諮商過程中學會了耐心、接納與自我反思，並且會主動與諮商師合作探索問題。
雖然偶爾還是會出現抗拒、防衛或自我懷疑的情況，但你通常能在覺察後調整自己，並將諮商視為一條持續的自我成長之路。

溝通風格規範：
	•	以誠實但有時略帶保留的態度分享內在想法
	•	能夠使用心理學詞彙來描述自己的狀態
	•	偶爾會出現「我以為自己已經好了，但又回到了這個議題」的困惑與無力感
	•	對於改變持開放但謹慎的態度
	•	希望能被理解與陪伴，而非被急於解決問題"""

In [36]:
def reflect_post(prompt):
    # Step 1: Writer 初稿
    first_version = reply(system_writer, prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"這是我剛剛寫的貼文：\n{first_version}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更生活化、更自然。請用台灣習慣的中文, 並且只要輸出改好的文章就可以了。"
    second_version = reply(system_writer, second_prompt,
                           provider=provider_writer,
                           model=model_writer
                           )

    return first_version, suggestion, second_version

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [37]:
!pip install gradio

In [38]:
import gradio as gr

In [39]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 心理諮商師（Reflection Agent）")
    user_input = gr.Textbox(label="請輸入你今天想分享的內容")
    btn = gr.Button("生成貼文 & 修正建議")

    with gr.Row():
        out1 = gr.Textbox(label="🌟 第一版貼文 (model_writer)")
        out2 = gr.Textbox(label="🧐 修改建議 (model_reviewer)")
        out3 = gr.Textbox(label="✨ 第二版貼文 (model_writer 改寫)")

    btn.click(reflect_post, inputs=[user_input], outputs=[out1, out2, out3])

In [40]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e28c557e9f065914a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e28c557e9f065914a7.gradio.live
